In [ ]:
import os
import sys
# Make sure all code is in the PATH.
sys.path.append(
    os.path.normpath(
        os.path.join(
            os.environ["HOME"], "Projects", "cosine_neutral_loss", "src"
        )
    )
)

In [ ]:
import functools
import lzma
import re

import Levenshtein
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numba as nb
import numpy as np
import pandas as pd
import pyteomics.mgf
import seaborn as sns
import spectrum_utils.spectrum as sus
import tqdm.notebook as tqdm
from matplotlib.colors import LogNorm

import similarity

In [ ]:
# Plot styling.
plt.style.use(["seaborn-white", "seaborn-paper"])
plt.rc("font", family="sans-serif")
sns.set_palette(["#9e0059", "#6da7de", "#dee000"])
sns.set_context("paper", font_scale=1.)

In [ ]:
regex_non_alpha = re.compile(r"[^A-Za-z]+")


@functools.lru_cache(None)
def remove_mod(sequence):
    return regex_non_alpha.sub("", sequence)

In [ ]:
charges = (2, 3, 4)

# Read all spectra from the MGF.
# MassIVE-KB (version 2018-06-15) downloaded from
# https://massive.ucsd.edu/ProteoSAFe/static/massive-kb-libraries.jsp
spectra = []
filename = ("../data/external/LIBRARY_CREATION_AUGMENT_LIBRARY_TEST-82c0124b-"
            "download_filtered_mgf_library-main.mgf.xz")
with lzma.open(filename, "rt") as xz_in:
    with pyteomics.mgf.MGF(xz_in) as f_in:
        for spectrum_dict in tqdm.tqdm(f_in):
            if int(spectrum_dict["params"]["charge"][0]) in charges:
                spec = sus.MsmsSpectrum(
                    spectrum_dict["params"]["seq"],
                    float(spectrum_dict["params"]["pepmass"][0]),
                    int(spectrum_dict["params"]["charge"][0]),
                    spectrum_dict["m/z array"],
                    spectrum_dict["intensity array"],
                )
                spec.remove_precursor_peak(0.1, "Da")
                spectra.append(spec)

In [ ]:
# Extract the metadata (peptide sequence and charge).
sequences, charges, mzs = [], [], []
for spectrum in spectra:
    sequences.append(spectrum.identifier)
    charges.append(spectrum.precursor_charge)
    mzs.append(spectrum.precursor_mz)
metadata = pd.DataFrame({"sequence": sequences, "charge": charges, "mz": mzs})
metadata["sequence"] = metadata["sequence"].str.replace("I", "L")
metadata["sequence_no_mod"] = metadata["sequence"].apply(remove_mod)
metadata["sequence_len"] = metadata["sequence_no_mod"].apply(len)

In [ ]:
min_mass_diff = 4    # Da


@nb.njit
def generate_pairs_ptm(spectrum_indexes, sequences, sequences_no_mod, masses):
    for i in range(len(spectrum_indexes)):
        j = i + 1
        while (j < len(sequences) and
               sequences_no_mod[i] == sequences_no_mod[j]):
            if (sequences[i] != sequences[j] and
                    abs(masses[i] - masses[j]) > min_mass_diff):
                yield spectrum_indexes[i]
                yield spectrum_indexes[j]
            j += 1

In [ ]:
# Extract indexes for pairs of spectra whose peptides differ by a
# modification (PTM or amino acid substitution).
pairs = []
for charge in np.arange(
        metadata["charge"].min(),
        metadata["charge"].max() + 1,
    ):
    metadata_charge = (metadata[metadata["charge"] == charge]
                       .copy()
                       .sort_values("sequence_no_mod")
                       .reset_index())
    # Pairs that differ by (one or more) PTMs.
    pairs.append(
        np.fromiter(
            generate_pairs_ptm(
                metadata_charge["index"].values,
                nb.typed.List(metadata_charge["sequence"]),
                nb.typed.List(metadata_charge["sequence_no_mod"]),
                metadata_charge["mz"].values * charge,
            ),
            np.int32)
        .reshape((-1, 2))
    )
    # Pairs that differ by a single AA (subtitution, addition/deletion).
    metadata["sequence_len"] = metadata["sequence"].apply(len)
    metadata_charge = metadata_charge.sort_values("sequence_len")
    spectrum_indexes = metadata_charge["index"].values
    sequences = metadata_charge["sequence"].values
    sequence_lens = metadata_charge["sequence_len"].values
    for i in tqdm.tqdm(range(len(metadata_charge))):
        for j in range(i + 1, len(metadata_charge)):
            if sequence_lens[j] - sequence_lens[i] > 1:
                break
            elif Levenshtein.distance(
                sequences[i], sequences[j], score_cutoff=1
            ) == 1:
                pairs.append((spectrum_indexes[i], spectrum_indexes[j]))
pairs = np.vstack(pairs)

In [ ]:
print(f"Number of spectrum pairs: {pairs.shape[0]:,}")

In [ ]:
# Compute similarities between spectrum pairs.
fragment_mz_tolerance = 0.1

cosines, modified_cosines, neutral_losses = [], [], []
for i, j in tqdm.tqdm(pairs):
    cosines.append(
        similarity.cosine(spectra[i], spectra[j], fragment_mz_tolerance)[0]
    )
    modified_cosines.append(
        similarity.modified_cosine(
            spectra[i], spectra[j], fragment_mz_tolerance
        )[0]
    )
    neutral_losses.append(
        similarity.neutral_loss(
            spectra[i], spectra[j], fragment_mz_tolerance
        )[0]
    )
similarities = pd.DataFrame(
    {
        "cosine": cosines,
        "modified_cosine": modified_cosines,
        "neutral_loss": neutral_losses,
    }
)
similarities[["pair1", "pair2"]] = pairs
similarities["sequence1"] = metadata.loc[pairs[:, 0], "sequence"].values
similarities["sequence2"] = metadata.loc[pairs[:, 1], "sequence"].values
similarities["charge1"] = metadata.loc[pairs[:, 0], "charge"].values
similarities["charge2"] = metadata.loc[pairs[:, 1], "charge"].values
similarities["mz1"] = metadata.loc[pairs[:, 0], "mz"].values
similarities["mz2"] = metadata.loc[pairs[:, 1], "mz"].values
similarities["sequence_len"] = np.amin(
    (
        metadata.loc[pairs[:, 0], "sequence_len"],
         metadata.loc[pairs[:, 1], "sequence_len"]),
    axis=0
)
similarities.to_parquet("massivekb_peptide_mods.parquet")

In [ ]:
labels = np.asarray([
    ["cosine", "modified_cosine"],
    ["neutral_loss", "cosine"],
    ["neutral_loss", "modified_cosine"]
])

mosaic = """
11111.
222223
222223
222223
222223
222223
"""

bins = 100
tick_locators = mticker.FixedLocator(np.arange(0, bins + 1, bins / 4))
tick_labels = np.asarray([f"{a:.2f}" for a in np.arange(0, 1.01, 0.25)])

with sns.plotting_context("paper", font_scale=1.6):
    fig = plt.figure(constrained_layout=True, figsize=(7.2 * 2, 7.2 / 1.618))
    left, middle, right = fig.subfigures(nrows=1, ncols=3)
    axes_left = left.subplot_mosaic(mosaic)
    axes_middle = middle.subplot_mosaic(mosaic)
    axes_right = right.subplot_mosaic(mosaic)
    cbar_ax = fig.add_axes([1.03, 0.25, 0.02, 0.5])

    for i, (axes, (xlabel, ylabel)) in enumerate(
        zip([axes_left, axes_middle, axes_right], labels)
    ):
        # Plot heatmaps.
        hist, _, _ = np.histogram2d(
            similarities[xlabel],
            similarities[ylabel],
            bins=bins,
            range=[[0, 1], [0, 1]],
        )
        hist /= len(similarities)
        heatmap = sns.heatmap(
            np.rot90(hist),
            vmin=0.0,
            vmax=0.01,
            cmap="viridis",
            cbar=i == 2,
            cbar_kws={"format": mticker.StrMethodFormatter("{x:.3%}")},
            cbar_ax=cbar_ax if i == 2 else None,
            square=True,
            xticklabels=False,
            yticklabels=False,
            ax=axes["2"],
            norm=LogNorm(),
        )
        axes["2"].yaxis.set_major_locator(tick_locators)
        axes["2"].set_yticklabels(tick_labels[::-1])
        axes["2"].xaxis.set_major_locator(tick_locators)
        axes["2"].set_xticklabels(tick_labels)
        for _, spine in heatmap.spines.items():
            spine.set_visible(True)
        axes["2"].set_xlabel(xlabel.replace("_", " ").capitalize())
        axes["2"].set_ylabel(ylabel.replace("_", " ").capitalize())

        axes["2"].plot(
            [0, bins], [bins, 0], color="black", linestyle="dashed"
        )

        sns.despine(ax=axes["2"])

        # Plot density plots.
        sns.kdeplot(
            data=similarities,
            x=xlabel,
            clip=(0, 1),
            legend=True,
            color="black",
            fill=True,
            ax=axes["1"],
        )
        axes["1"].set_xlim(0, 1)
        axes["1"].xaxis.set_ticklabels([])
        axes["1"].yaxis.set_major_locator(tick_locators)
        axes["1"].set_yticks([])
        sns.despine(ax=axes["1"], left=True)
        sns.kdeplot(
            data=similarities,
            y=ylabel,
            clip=(0, 1),
            legend=True,
            color="black",
            fill=True,
            ax=axes["3"],
        )
        axes["3"].set_ylim(0, 1)
        axes["3"].yaxis.set_ticklabels([])
        axes["3"].xaxis.set_major_locator(tick_locators)
        axes["3"].set_xticks([])
        sns.despine(ax=axes["3"], bottom=True)
        for ax in [axes[c] for c in "13"]:
            ax.set_xlabel("")
            ax.set_ylabel("")
            
    cbar_ax.set_ylabel("Proportion of pairs")
    cbar_ax.yaxis.set_label_position("left")
    cbar_ax.spines["outline"].set(visible=True, lw=.8, edgecolor="black")
            
    plt.savefig("massivekb_peptide_mods.png", dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()

In [ ]:
print(
    f"Spectrum pairs where neutral loss outperforms cosine: "
    f"{(similarities['neutral_loss'] > similarities['cosine']).sum() / len(similarities):.1%}"
)
print(
    f"Spectrum pairs where neutral loss outperforms modified cosine: "
    f"{(similarities['neutral_loss'] > similarities['modified_cosine']).sum() / len(similarities):.1%}"
)

In [ ]:
@nb.njit
def get_mod_pos(sequence1, sequence2):
    i = 0
    for aa1, aa2 in zip(sequence1, sequence2):
        if aa1 != aa2:
            return i
        elif aa1 not in "+-1234567890.":
            i += 1
    return i


similarities["mod_pos"] = [
    get_mod_pos(sequence1, sequence2)
    for sequence1, sequence2 in zip(
        similarities["sequence1"],
        similarities["sequence2"],
    )
]
similarities["rel_mod_pos"] = (
    similarities["mod_pos"] / similarities["sequence_len"]
)

In [ ]:
similarities["mod_interval"] = pd.cut(
    similarities["rel_mod_pos"],
    5,
    labels=["0.0–0.2", "0.2–0.4", "0.4–0.6", "0.6–0.8", "0.8–1.0"]
)
mods_location = pd.melt(
    similarities,
    id_vars="mod_interval",
    value_vars=["cosine", "neutral_loss", "modified_cosine"],
)

width = 3.5
height = width / 1.618
fig, ax = plt.subplots(figsize=(7.2, height))

sns.violinplot(
    data=mods_location,
    x="mod_interval",
    y="value",
    hue="variable",
    hue_order=["cosine", "neutral_loss", "modified_cosine"],
    cut=0,
    scale="width",
    scale_hue=False,
    ax=ax,
)
ax.set_xlabel("Relative modification position")
ax.set_ylabel("Similarity score")
for label in ax.legend().get_texts():
    label.set_text(label.get_text().replace("_", " ").capitalize())
sns.move_legend(
    ax,
    "lower center",
    bbox_to_anchor=(.5, 1),
    ncol=3,
    title=None,
    frameon=False,
)

sns.despine(ax=ax)

plt.savefig(
    "massivekb_peptide_mods_location_score.png", dpi=300, bbox_inches="tight"
)
plt.show()
plt.close()